In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import random
import os
import json
from urllib.request import Request, urlopen 

In [2]:
# web crawl from github
driver = webdriver.Chrome(ChromeDriverManager().install())
github_account = {}




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\17163\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
def get_git(url):
    driver.get(url)
    github_account["login"] = driver.find_element_by_xpath('//meta[@property="profile:username"]').get_attribute('content')
    github_account["id"] = driver.find_element_by_class_name('js-site-search-form').get_attribute('data-scope-id')
    github_account["html_url"] = driver.find_element_by_xpath('//meta[@property="og:url"]').get_attribute('content')
    github_account["type"] = driver.find_element_by_class_name('js-site-search-form').get_attribute('data-scope-type') 
#     github_account["location"] = driver.find_element_by_xpath('//span[@itemprop="location"]').get_attribute('title')
    return github_account

In [8]:
# web crawl from monster
head = "https://www.monster.com/"
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

# job_titles = ["data+scientist"]
job_titles = ["data+scientist"]

In [11]:
def get_soup(url):
    """
    This function get the beautifulsoup object of a webpage.

    Args:
        url (str): the link string of webpage

    Returns:
        soup (obj): beautifulsoup object
    """
    request = Request(url, headers={'User-Agent': 'Resistance is futile'})
    response = urlopen(request)
    return BeautifulSoup(response, "html.parser")

def get_jobs_of_title(job_title):
    """
    Args:
        job_title (str): example: 'data+scientist'

    Returns:
    """

    #needed to be changed
    num_pages = 1 #number of pages to scrape
    page_gap_min = 3 #min sleep time between pages
    page_gap_max = 5 #max sleep time between pages
    job_per_page = 50 #number of jobs in one page
    job_gap_min = 5 #min sleep time between jobs
    job_gap_max = 6 #max sleep time between jobs

    for i in range(num_pages): 
        #sleep between each call
        gap = random.uniform(page_gap_min,page_gap_max) 
        time.sleep(gap)

        #each page contains 50 jobs
        tail = "jobs?q={0}&sort=date&limit={1}".format(job_title,job_per_page)
        if i>0:
            tail += "&start={0}".format(i*job_per_page)

        #get link to joblist page
        url = head+tail 
         
        #get links to webpages of jobs on the joblist
        job_page_links = get_job_links_from_page(url)

        for job_page_link in job_page_links:
            gap = random.uniform(job_gap_min,job_gap_max) 
            time.sleep(gap)
            data = get_info_from_job_page(job_page_link)

            print(json.dumps(data))

def get_job_links_from_page(url):
    """
    This function gets the links of the jobs on the joblist page.

    Args:
        url (str): link to joblist page

    Returns:
        job_page_links (list): list of links to the webpages of the jobs
    """

    job_page_links = []
    soup = get_soup(url)
    for item in soup.find_all("a", href=True):
        if '/rc/clk?jk=' in str(item) and 'fccid=' in str(item):
            link = item['href'].split("clk?")[1]
            job_page_links.append(head+'viewjob?'+link)
    return job_page_links

def get_info_from_job_page(url):
    """
    This function get all the useful info from the job webpage.

    Args:
        url (str): link to job webpage

    Returns:
        data (dict): dictionary with keywords: 
                     time_stamp, original_link, job_title, location, company, description
    """
    soup = get_soup(url)
    data = {}
    time_str = soup.find('div',class_='result-link-bar').find('span').getText()
    try:
        data["time_stamp"] = get_timestamp(time_str).strftime("%d-%m-%Y %H:%M")
        data["job_title"] = soup.find('b', class_='jobtitle').getText()
        data["location"] = soup.find('span', class_='location').getText()
        data["company"] = soup.find('span', class_='company').getText()
        data["description"] = soup.find('td',class_='snip').find('div').getText()

        re_link = soup.find('a',class_='sl ws_label')['href'].split("&from=")[0]
        re_link = head[:-1]+re_link
        data["original_link"] = get_original_link(re_link)
    except:
        pass
    return data

def get_timestamp(time_str):
    """
    Calculate the timestamp from the time string.
    
    Args:
        time_str (str): time string, like '2 hours ago'

    Returns:
        time_stamp (obj): timestamp object
    """
    if 'hour' in time_str:
        lag = int(time_str.split('hour')[0])
        delta = timedelta(hours=lag)
        now = datetime.utcnow().replace(second=0,minute=0)
        return now-delta
    else:
        return -1

def get_original_link(url):
    """
    Get the original link of the job description.
    
    Args:
        url (str): the link in Indeed database

    Returns:
        url (str): the original link to the job description
    """
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(2)
    original_url = driver.current_url
    driver.quit()
    return original_url

In [10]:
get_soup("https://www.monster.com/")

<!DOCTYPE html>
<html lang="en-us"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta content="Gatsby 2.32.13" name="generator"/><title data-react-helmet="true">Monster Jobs - Job Search, Career Advice &amp; Hiring Resources | Monster.com</title><link as="font" crossorigin="anonymous" data-react-helmet="true" href="/static/Proxima-Nova-Regular-2000f9d8a8df2554f0c0994f44057524.woff2" rel="preload" type="font/woff"/><link as="font" crossorigin="anonymous" data-react-helmet="true" href="/static/Proxima-Nova-SemiBold-2b6fce92d9bb4f455f961b9c9df136b9.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" data-react-helmet="true" href="/static/Proxima-Nova-Bold-d31288e9360bd8c887f67632bd998983.woff2" rel="preload" type="font/woff2"/><link data-react-helmet="true" href="https://www.monster.com" rel="canonical"/><link data-react-helmet="true" hre